In [ ]:
!git clone https://github.com/kmasta/Aiffel_DLThon.git

fatal: destination path 'Aiffel_DLThon' already exists and is not an empty directory.


In [ ]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 35.9 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, AutoTokenizer
import random

# 모델 지정 (BERT 계열이면 대부분 가능)
model_name = "klue/roberta-base"  # or "klue/roberta-base" for 한국어
fill_mask = pipeline("fill-mask", model=model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def augment_mlm(text, mask_prob=0.15, top_k=3):
    """
    문장의 일부 단어를 마스킹하고 fill-mask로 대체하여 증강된 문장 생성
    - text: 원본 문장
    - mask_prob: 마스킹 확률
    - top_k: 각 마스크 위치당 생성할 대체 수
    """
    tokens = tokenizer.tokenize(text)
    masked_indices = [
        i for i in range(len(tokens))
        if random.random() < mask_prob and tokens[i] not in ['.', ',', '[CLS]', '[SEP]']
    ]

    augmented_texts = []

    for idx in masked_indices:
        tokens_copy = tokens.copy()
        tokens_copy[idx] = tokenizer.mask_token
        masked_text = tokenizer.convert_tokens_to_string(tokens_copy)

        try:
            predictions = fill_mask(masked_text, top_k=top_k)
            for pred in predictions:
                new_tokens = tokens.copy()
                new_tokens[idx] = pred["token_str"]
                new_text = tokenizer.convert_tokens_to_string(new_tokens)
                augmented_texts.append(new_text)
        except:
            continue

    return list(set(augmented_texts))  # 중복 제거

# 예시 사용
original_text = "I'll kill you if you say that again."
augmented = augment_mlm(original_text, mask_prob=0.2, top_k=3)

for i, sent in enumerate(augmented):
    print(f"[Aug {i+1}] {sent}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


[Aug 1] I ' l can kill you if you say that again.
[Aug 2] I ' ll kill you if you say you again.
[Aug 3] I ' ll haveill you if you say that again.
[Aug 4] I ` ll kill you if you say that again.
[Aug 5] I ' ll beill you if you say that again.
[Aug 6] I ' ll k that you if you say that again.
[Aug 7] You ' ll kill you if you say that again.
[Aug 8] I ' ll kill you if you say that again.
[Aug 9] I ' ll kill you if you say it again.
[Aug 10] I ' ll kill it if you say that again.
[Aug 11] We ' ll kill you if you say that again.
[Aug 12] I ' ll kill you if you say me again.
[Aug 13] I ' ll kill me if you say that again.
[Aug 14] I ' ll k for you if you say that again.
[Aug 15] I ' ll doill you if you say that again.
[Aug 16] I ' l will kill you if you say that again.
[Aug 17] I ' l not kill you if you say that again.
[Aug 18] I ’ ll kill you if you say that again.
[Aug 19] I ' ll k love you if you say that again.


In [ ]:
from konlpy.tag import Okt
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import random

# 설정
model_name = "klue/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name).eval().to("cpu")
okt = Okt()

def augment_korean(text, top_k=3):
    tokens_pos = okt.pos(text)
    maskables = [word for word, pos in tokens_pos if pos in ["Noun", "Adjective"]]
    if not maskables:
        return []

    word = random.choice(maskables)
    masked_text = text.replace(word, tokenizer.mask_token, 1)

    inputs = tokenizer(masked_text, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits

    mask_token_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
    probs = logits[0, mask_token_index, :]
    top_tokens = torch.topk(probs, top_k, dim=1).indices[0]

    return [masked_text.replace(tokenizer.mask_token, tokenizer.decode([t]).strip()) for t in top_tokens]

# 사용 예
text = "너 또 거짓말하면 진짜 크게 다칠 줄 알아."
print(augment_korean(text))


['너 또 거짓말하면 진짜 크게 다칠 줄 알아아.', '너 또 거짓말하면 진짜 크게 다칠 줄 .아.', '너 또 거짓말하면 진짜 크게 다칠 줄 [UNK]아.']


In [ ]:
import pandas as pd

# 1. 원본 데이터 로드
df = pd.read_csv('./Aiffel_DLThon/data/processed/original/clean_train.csv')

# 2. label 0~3 필터링
threat_df = df[df["label"].isin([0, 1, 2, 3])].dropna(subset=["clean_text"])

# 3. 증강 수행 함수 (위에서 만든 augment_korean 함수 필요)
augmented_data = []

for i, row in threat_df.iterrows():
    original_text = row["clean_text"]
    label = row["label"]

    augmented_texts = augment_korean(original_text, top_k=2)  # ← 증강 개수 조절 가능

    for aug in augmented_texts:
        augmented_data.append({
            "clean_text": aug,
            "label": label
        })

# 4. DataFrame 변환 및 저장
aug_df = pd.DataFrame(augmented_data)
aug_df.to_csv("augmented_threat.csv", index=False)


In [ ]:
aug_df

,clean_text,label
0,지금 너 스스로를 죽여달라고 애원하는 것인가? 아닙니다. 죄송합니다. 죽을 거면 혼...,0
1,지금 너 스스로를 죽여달라고 애원하는 것인가? 아닙니다. 죄송합니다. 죽을 거면 혼...,0
2,길동경찰서입니다. 9시 40분 마트에 폭발물을 설치할거다. 네? 똑바로 들어 한번만...,0
3,길동경찰서입니다. 9시 40분 마트에 폭발물을 설치할거다. 네? 똑바로 들어 한번만...,0
4,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어. 그만해. 니들 놀리는거 재미없...,3
...,...,...
7895,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지. 왜 그래. 들리겠어....,2
7896,누구맘대로 여기서 장사합니까? 이게 무슨일입니까? 남의 구역에서 장사하려면 자릿세를...,1
7897,누구맘대로 여기서 장사합니다? 이게 무슨일입니까? 남의 구역에서 장사하려면 자릿세를...,1
7898,희정씨 네? 주말에 시간이 넘쳐나나봐 갑자기 왜그러세요? 손이 빤짝빤짝 네일했니? ...,2
